In [1]:
# test
import serial
import csv
import time

# open the serial port for communication with Arduino
with serial.Serial('/dev/ttyACM0', 9600, timeout=1) as arduino:
    time.sleep(0.1) #wait for serial to open

    # create a CSV file to store altitude data
    with open('/home/sraq/Desktop/Exam/alti.csv', 'w') as csvFile:
        if arduino.isOpen():
            print("{} connected!".format(arduino.port))
            writer = csv.writer(csvFile)
            writer.writerow(['Altitude'])
              
            # continuously read altitude data from Arduino and write it to CSV file
            try:
                while True:
                     #serial1= serial.Serial("/dev/ttyACM3",9600)
                     if  arduino.inWaiting()>0:
                            answer=arduino.readline().decode('utf-8')
                            print(answer)
                            writer.writerow([answer])
                            # csvFile.write(answer)
                            # serial1.write(answer.encode())
                            arduino.flushInput() #remove data after reading
            except KeyboardInterrupt:
                            print("KeyboardInterrupt has been caught.")

ModuleNotFoundError: No module named 'serial'

In [5]:
# test

import serial
import csv
import time

# open the serial port for communication with Arduino
with serial.Serial('/dev/ttyUSB0', 9600, timeout=1) as arduino:
    time.sleep(0.1) #wait for serial to open
    if arduino.isOpen():
        print("{} connected!".format(arduino.port))
        while True:
            # create a CSV file to store altitude data
            with open('/home/sraq/Desktop/Exam/alti1.csv', 'a',newline='') as csvFile:
                    writer = csv.writer(csvFile)
                    #continuously read altitude data from Arduino and write it to CSV file
                    try:
                            if  arduino.inWaiting()>0:
                                    answer=arduino.readline().decode('utf-8').strip()
                                    print(answer)
                                    writer.writerow([answer])
                                    arduino.flushInput() #remove data after reading
                    except KeyboardInterrupt:
                                    print("KeyboardInterrupt has been caught.")

ModuleNotFoundError: No module named 'serial'

In [6]:
#part 2

import socket
import csv
import struct

UDP_IP = "192.168.42.168" # replace with server IP
UDP_IP2 = "192.168.42.29"
UDP_PORT = 54321 # replace with port number
UDP_PORT2 = 54322
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))

sock2 = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
# sock2.bind((UDP_IP, UDP_PORT2))


# while True:
#     data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
#     print("Fuel 1: ", data.decode())


while True:
    # create a CSV file to store fuel data
    with open('/home/sraq/Desktop/Exam/fuel.csv', 'a',newline='') as csvFile:
            writer = csv.writer(csvFile)
            #continuously read fuel 1 data from Arduino and write it to CSV file
            try:
                data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
                received_value = data.decode()
                
                fuel_1, fuel_2 = received_value.split(',')  # values are separated by a comma
                fuel_1 = float(fuel_1.strip())
                fuel_2 = float(fuel_2.strip())

                print("Fuel 1: {}, Fuel 2: {}".format(fuel_1, fuel_2))

                writer.writerow([fuel_1,fuel_2])

                received_value = struct.pack('f,f',fuel_1,fuel_2)
                sock2.sendto(received_value, (UDP_IP2, UDP_PORT2))

            except KeyboardInterrupt:
                print("KeyboardInterrupt has been caught.")

OSError: [WinError 10049] The requested address is not valid in its context

In [9]:
# part 3 it receives the data and save it to respective CSV files and simultaneously sent it to respective microcontrollers.

import serial
import time
import socket
import csv
import struct
import datetime
import json

rpi = "192.168.42.168" # server IP
m1 = "192.168.42.123"
m2 = "192.168.42.29"

m1_port = 54321 # port number
m2_port = 54322

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((rpi, m1_port))

sock2 = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP


# open the serial port for communication with Arduino
with serial.Serial('/dev/ttyACM1', 9600, timeout=1) as arduino1:
    time.sleep(0.1) #wait for serial to open
    if arduino1.isOpen():
         print("{} connected!".format(arduino1.port))


    with serial.Serial('/dev/ttyACM0',9600,timeout = 1) as arduino2:
            time.sleep(0.1)
            if arduino2.isOpen():
                print("{} connected!".format(arduino2.port))

                while True:    
                    try:
                        user = input("Enter 'S' to to run the level flight system: ")
                        if (user == 's' or user == 'S'):
                            
                            ack = "S"
                            arduino1.write(ack.encode('utf-8')) #sending Acknowledgement to arduino 1 so that it can send the next amplitude data    
                            sock.sendto("S".encode(), (m1,m1_port)) 
                            while True:
                                # create a CSV file to store altitude data
                                with open('/home/sraq/Desktop/Exam/alti1.csv', 'a',newline='') as altitude:
                                        writer = csv.writer(altitude)
                                        #continuously read altitude data from Arduino and write it to CSV file
                                        try:
                                            answer=arduino1.readline().decode('utf-8').strip() # receiving amplitude data from arduino 1
                                            # print("Amplitude: ",answer)
                                            writer.writerow([answer])   #writing to csv file
                                            
                                            arduino1.flushInput() #remove data after reading
                                        except KeyboardInterrupt:
                                            print("KeyboardInterrupt has been caught.")

                                # create a CSV file to store fuel data
                                with open('/home/sraq/Desktop/Exam/fuel.csv', 'a',newline='') as csvFile:
                                        writer = csv.writer(csvFile)
                                        #continuously read fuel 1 data from Arduino and write it to CSV file
                                        try:
                                            data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes and receiving fuel data from nodeMCU 1
                                            received_value = data.decode()
                                            
                                            fuel_1, fuel_2 = received_value.split(',')  # values are separated by a comma
                                            fuel_1 = float(fuel_1.strip())
                                            fuel_2 = float(fuel_2.strip())

                                            # print("Fuel 1: {}, Fuel 2: {}".format(fuel_1, fuel_2))

                                            writer.writerow([fuel_1,fuel_2])

                                            received_value = struct.pack('ff',fuel_1,fuel_2)

                                        except KeyboardInterrupt:
                                            print("KeyboardInterrupt has been caught.")

                                
                                current_time = datetime.datetime.now()
                                time_str = current_time.strftime("%S")

                                data = {}
                                data["time"] = time_str
                                data["fuel_1"] = fuel_1
                                data["fuel_2"] = fuel_2
                                data["altitude"] = answer
                                if (fuel_2< 20):
                                    print("Amplitude: {}, Fuel 1: {}, Fuel 2: {}, Time: {}, Warning fuel 2 < 20".format(answer,fuel_1,fuel_2,time_str))

                                else:
                                    print("Amplitude: {}, Fuel 1: {}, Fuel 2: {}, Time: {}".format(answer,fuel_1,fuel_2,time_str))
                                
                                json_data = json.dumps(data)
                                client_socket.sendto(json_data.encode(), (rpi, 8888))
                                
                                arduino2.write((str(answer)+"\n").encode('utf-8')) # sending amplitude data from arduino 2
                                sock2.sendto(received_value, (m2, m2_port)) #sending fuel data from nodeMCU 2
                                    
                                ack = "RS"
                                arduino1.write(ack.encode('utf-8')) #sending Acknowledgement to arduino 1 so that it can send the next amplitude data
                                
                                # ack = "RS"
                                # sock.sendto(struct.pack('{}s'.format(len(ack)),ack), (m1, m1_port))   # sending Acknowledgement to nodeMCU 1 so that it can send the next fuel data
                                sock.sendto("RS".encode(), (m1,m1_port))
                            
                            else:
                                raise ValueError("Invalid")
                    except ValueError as e:
                        print(e)

ModuleNotFoundError: No module named 'serial'

In [ ]:
# part 3, it receives the data and simultaneously sent it to respective microcontrollers.
 

import serial
import time
import socket
import csv
import struct
import datetime
import json

rpi = "192.168.42.168" # server IP
m1 = "192.168.42.123"
m2 = "192.168.42.29"

m1_port = 54321 # port number
m2_port = 54322

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((rpi, m1_port))

sock2 = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# open the serial port for communication with Arduino
with serial.Serial('/dev/ttyACM1', 9600, timeout=1) as arduino1:
    time.sleep(0.1) #wait for serial to open
    if arduino1.isOpen():
         print("{} connected!".format(arduino1.port))


    with serial.Serial('/dev/ttyACM0',9600,timeout = 1) as arduino2:
            time.sleep(0.1)
            if arduino2.isOpen():
                print("{} connected!".format(arduino2.port))

                while True:    
                    try:
                        user = input("Enter 'S' to to run the level flight system: ")
                        if (user == 's' or user == 'S'):
                            
                            ack = "S"
                            arduino1.write(ack.encode('utf-8')) #sending Acknowledgement to arduino 1 so that it can send the next amplitude data    
                            sock.sendto("S".encode(), (m1,m1_port)) 
                            
                            while True:
                               
                                answer=arduino1.readline().decode('utf-8').strip() # receiving amplitude data from arduino 1
                                
                                data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes and receiving fuel data from nodeMCU 1
                                received_value = data.decode()
                                fuel_1, fuel_2 = received_value.split(',')  # values are separated by a comma
                                fuel_1 = float(fuel_1.strip())
                                fuel_2 = float(fuel_2.strip())
                                received_value = struct.pack('ff',fuel_1,fuel_2)
                    
                                current_time = datetime.datetime.now()
                                time_str = current_time.strftime("%S")

                                data = {}
                                data["time"] = time_str
                                data["fuel_1"] = fuel_1
                                data["fuel_2"] = fuel_2
                                data["altitude"] = answer
                                if (fuel_2< 20):
                                    print("Amplitude: {}, Fuel 1: {}, Fuel 2: {}, Time: {}, Warning fuel 2 < 20".format(answer,fuel_1,fuel_2,time_str))

                                else:
                                    print("Amplitude: {}, Fuel 1: {}, Fuel 2: {}, Time: {}".format(answer,fuel_1,fuel_2,time_str))
                                
                                json_data = json.dumps(data)
                                client_socket.sendto(json_data.encode(), (rpi, 8888))
                                
                                arduino2.write((str(answer)+"\n").encode('utf-8')) # sending amplitude data to arduino 2
                                sock2.sendto(received_value, (m2, m2_port)) #sending fuel data to nodeMCU 2
                                    
                                ack = "RS"
                                arduino1.write(ack.encode('utf-8')) #sending Acknowledgement to arduino 1 so that it can send the next amplitude data
                                sock.sendto("RS".encode(), (m1,m1_port))
                            
                        else:
                            raise ValueError("Invalid")
                    except ValueError as e:
                        print(e)